In [1]:
def get_anchors(anchors_path):
    '''loads the anchors from a file'''
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape(-1, 2)

In [2]:
def get_classes(classes_path):
    '''loads the classes'''
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

In [ ]:
def get_dataset(annotation_file, shuffle=True):
    with open(annotation_file) as f:
        lines = f.readlines()
        lines = [line.strip() for line in lines]

    if shuffle:
        np.random.seed(int(time.time()))
        np.random.shuffle(lines)
        #np.random.seed(None)

    return lines

In [ ]:
def load_eval_model(model_path):
    # support of tflite model
    if model_path.endswith('.tflite'):
        from tensorflow.lite.python import interpreter as interpreter_wrapper
        model = interpreter_wrapper.Interpreter(model_path=model_path)
        model.allocate_tensors()
        model_format = 'TFLITE'

    # normal keras h5 model
    elif model_path.endswith('.h5'):
        custom_object_dict = get_custom_objects()

        model = load_model(model_path, compile=False, custom_objects=custom_object_dict)
        model_format = 'H5'
        K.set_learning_phase(0)
    else:
        raise ValueError('invalid model file')

    return model, model_format


In [9]:
def annotation_parse(annotation_lines, class_names):
    '''
    parse annotation lines to get image dict and ground truth class dict

    image dict would be like:
    annotation_records = {
        '/path/to/000001.jpg': {'100,120,200,235':'dog', '85,63,156,128':'car', ...},
        ...
    }

    ground truth class dict would be like:
    classes_records = {
        'car': [
                ['000001.jpg','100,120,200,235'],
                ['000002.jpg','85,63,156,128'],
                ...
               ],
        ...
    }
    '''
    annotation_records = OrderedDict()
    classes_records = OrderedDict({class_name: [] for class_name in class_names})

    for line in annotation_lines:
        box_records = {}
        image_name = line.split(' ')[0]
        boxes = line.split(' ')[1:]
        for box in boxes:
            # strip box coordinate and class
            class_name = class_names[int(box.split(',')[-1])]
            coordinate = ','.join(box.split(',')[:-1])
            box_records[coordinate] = class_name
            # append or add ground truth class item
            record = [os.path.basename(image_name), coordinate]
            if class_name in classes_records:
                classes_records[class_name].append(record)
            else:
                classes_records[class_name] = list([record])
        annotation_records[image_name] = box_records

    return annotation_records, classes_records

In [ ]:
def yolo_predict_keras(model, image, anchors, num_classes, model_image_size, conf_threshold, elim_grid_sense,
                       v5_decode):
    image_data = preprocess_image(image, model_image_size)
    # origin image shape, in (height, width) format
    image_shape = tuple(reversed(image.size))

    prediction = model.predict([image_data])
    if len(anchors) == 5:
        # YOLOv2 use 5 anchors
        pred_boxes, pred_classes, pred_scores = yolo2_postprocess_np(prediction, image_shape, anchors, num_classes,
                                                                     model_image_size, max_boxes=100,
                                                                     confidence=conf_threshold,
                                                                     elim_grid_sense=elim_grid_sense)
    else:
        if v5_decode:
            pred_boxes, pred_classes, pred_scores = yolo5_postprocess_np(prediction, image_shape, anchors, num_classes,
                                                                         model_image_size, max_boxes=100,
                                                                         confidence=conf_threshold,
                                                                         elim_grid_sense=True)  # enable "elim_grid_sense" by default
        else:
            pred_boxes, pred_classes, pred_scores = yolo3_postprocess_np(prediction, image_shape, anchors, num_classes,
                                                                         model_image_size, max_boxes=100,
                                                                         confidence=conf_threshold,
                                                                         elim_grid_sense=elim_grid_sense)

    return pred_boxes, pred_classes, pred_scores

In [ ]:
def get_prediction_class_records(model, model_format, annotation_records, anchors, class_names, model_image_size,
                                 conf_threshold, elim_grid_sense, v5_decode, save_result):
    '''
    Do the predict with YOLO model on annotation images to get predict class dict

    predict class dict would contain image_name, coordinary and score, and
    sorted by score:
    pred_classes_records = {
        'car': [
                ['000001.jpg','94,115,203,232',0.98],
                ['000002.jpg','82,64,154,128',0.93],
                ...
               ],
        ...
    }
    '''

    # create txt file to save prediction result, with
    # save format as annotation file but adding score, like:
    #
    # path/to/img1.jpg 50,100,150,200,0,0.86 30,50,200,120,3,0.95
    #
    os.makedirs('result', exist_ok=True)
    result_file = open(os.path.join('result', 'detection_result.txt'), 'w')

    pred_classes_records = OrderedDict()
    pbar = tqdm(total=len(annotation_records), desc='Eval model')
    for (image_name, gt_records) in annotation_records.items():
        image = Image.open(image_name)
        if image.mode != 'RGB':
            image = image.convert('RGB')
        image_array = np.array(image, dtype='uint8')

        # normal keras h5 model
        if model_format == 'H5':
            pred_boxes, pred_classes, pred_scores = yolo_predict_keras(model, image, anchors, len(class_names),
                                                                       model_image_size, conf_threshold,
                                                                       elim_grid_sense, v5_decode)
        else:
            raise ValueError('invalid model format')

        # print('Found {} boxes for {}'.format(len(pred_boxes), image_name))
        pbar.update(1)

        # save prediction result to txt
        result_file.write(image_name)
        for box, cls, score in zip(pred_boxes, pred_classes, pred_scores):
            xmin, ymin, xmax, ymax = box
            box_annotation = " %d,%d,%d,%d,%d,%f" % (
                xmin, ymin, xmax, ymax, cls, score)
            result_file.write(box_annotation)
        result_file.write('\n')
        result_file.flush()

        if save_result:

            gt_boxes, gt_classes, gt_scores = transform_gt_record(gt_records, class_names)

            result_dir = os.path.join('result', 'detection')
            os.makedirs(result_dir, exist_ok=True)
            colors = get_colors(class_names)
            image_array = draw_boxes(image_array, gt_boxes, gt_classes, gt_scores, class_names, colors=None,
                                     show_score=False)
            image_array = draw_boxes(image_array, pred_boxes, pred_classes, pred_scores, class_names, colors)
            image = Image.fromarray(image_array)
            # here we handle the RGBA image
            if (len(image.split()) == 4):
                r, g, b, a = image.split()
                image = Image.merge("RGB", (r, g, b))
            image.save(os.path.join(result_dir, image_name.split(os.path.sep)[-1]))

        # Nothing detected
        if pred_boxes is None or len(pred_boxes) == 0:
            continue

        for box, cls, score in zip(pred_boxes, pred_classes, pred_scores):
            pred_class_name = class_names[cls]
            xmin, ymin, xmax, ymax = box
            coordinate = "{},{},{},{}".format(xmin, ymin, xmax, ymax)

            # append or add predict class item
            record = [os.path.basename(image_name), coordinate, score]
            if pred_class_name in pred_classes_records:
                pred_classes_records[pred_class_name].append(record)
            else:
                pred_classes_records[pred_class_name] = list([record])

    # sort pred_classes_records for each class according to score
    for pred_class_list in pred_classes_records.values():
        pred_class_list.sort(key=lambda ele: ele[2], reverse=True)

    pbar.close()
    result_file.close()
    return pred_classes_records

In [10]:
def get_dataset(annotation_file, shuffle=True):
    with open(annotation_file) as f:
        lines = f.readlines()
        lines = [line.strip() for line in lines]

    if shuffle:
        np.random.seed(int(time.time()))
        np.random.shuffle(lines)
        #np.random.seed(None)

    return lines

In [ ]:
def eval_AP(model, model_format, annotation_lines, anchors, class_names, model_image_size, eval_type, iou_threshold,
            conf_threshold, elim_grid_sense, v5_decode, save_result, class_filter=None):
    '''
    Compute AP for detection model on annotation dataset
    '''
    annotation_records, gt_classes_records = annotation_parse(annotation_lines, class_names)
    pred_classes_records = get_prediction_class_records(model, model_format, annotation_records, anchors, class_names,
                                                        model_image_size, conf_threshold, elim_grid_sense, v5_decode,
                                                        save_result)
    AP = 0.0

    if eval_type == 'VOC':
        AP, APs = compute_mAP_PascalVOC(annotation_records, gt_classes_records, pred_classes_records, class_names,
                                        iou_threshold)

        if class_filter is not None:
            get_filter_class_mAP(APs, class_filter)

    elif eval_type == 'COCO':
        AP, _ = compute_AP_COCO(annotation_records, gt_classes_records, pred_classes_records, class_names, class_filter)
        # get AP for different scale: small, medium, large
        scale_gt_classes_records = get_scale_gt_dict(gt_classes_records, class_names)
        compute_AP_COCO_Scale(annotation_records, scale_gt_classes_records, pred_classes_records, class_names)
    else:
        raise ValueError('Unsupported evaluation type')

    return AP

In [7]:
import os
os.makedirs('result', exist_ok=True)
result_file = open(os.path.join('result', 'detection_result.txt'), 'w')

In [11]:
annotation_lines = get_dataset('2007_test.txt', shuffle=False)
annotation_lines[:2]

['C:\\Users\\xia\\Documents\\datasets\\VOCdevkit/VOC2007/JPEGImages/000001.jpg 48,240,195,371,11 8,12,352,498,14',
 'C:\\Users\\xia\\Documents\\datasets\\VOCdevkit/VOC2007/JPEGImages/000002.jpg 139,200,207,301,18']

In [14]:
from collections import OrderedDict
classes = ["aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", "chair", "cow", "diningtable", "dog", "horse", "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"]
annotation_records, gt_classes_records = annotation_parse(annotation_lines,classes)

In [16]:
from tqdm import tqdm
pbar = tqdm(total=len(annotation_records), desc='Eval model')

Eval model:   0%|                                                                             | 0/4952 [00:00<?, ?it/s]

In [31]:
from PIL import Image
import numpy as np
model_image_size = (416,416)

for (image_name, gt_records) in annotation_records.items():
    print(image_name)
    image = Image.open(image_name)
    if image.mode != 'RGB':
            image = image.convert('RGB')
#     image.show()
    image_array = np.array(image, dtype='uint8')
    image_data = preprocess_image(image, model_image_size)
    image_shape = tuple(reversed(image.size))
    print(image_data)
    break

C:\Users\xia\Documents\datasets\VOCdevkit/VOC2007/JPEGImages/000001.jpg
[[[[0.5019608 0.5019608 0.5019608]
   [0.5019608 0.5019608 0.5019608]
   [0.5019608 0.5019608 0.5019608]
   ...
   [0.5019608 0.5019608 0.5019608]
   [0.5019608 0.5019608 0.5019608]
   [0.5019608 0.5019608 0.5019608]]

  [[0.5019608 0.5019608 0.5019608]
   [0.5019608 0.5019608 0.5019608]
   [0.5019608 0.5019608 0.5019608]
   ...
   [0.5019608 0.5019608 0.5019608]
   [0.5019608 0.5019608 0.5019608]
   [0.5019608 0.5019608 0.5019608]]

  [[0.5019608 0.5019608 0.5019608]
   [0.5019608 0.5019608 0.5019608]
   [0.5019608 0.5019608 0.5019608]
   ...
   [0.5019608 0.5019608 0.5019608]
   [0.5019608 0.5019608 0.5019608]
   [0.5019608 0.5019608 0.5019608]]

  ...

  [[0.5019608 0.5019608 0.5019608]
   [0.5019608 0.5019608 0.5019608]
   [0.5019608 0.5019608 0.5019608]
   ...
   [0.5019608 0.5019608 0.5019608]
   [0.5019608 0.5019608 0.5019608]
   [0.5019608 0.5019608 0.5019608]]

  [[0.5019608 0.5019608 0.5019608]
   [0.5019

In [23]:
pbar.update(1)

Eval model:   0%|                                                                | 1/4952 [04:11<346:14:43, 251.76s/it]

True

In [25]:
from data_utils import *

In [28]:
model_image_size = (416,416)
tuple(reversed(model_image_size))

(416, 416)

In [43]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Conv2D, DepthwiseConv2D, Concatenate, MaxPooling2D, BatchNormalization, Activation, UpSampling2D, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
def mish(x):
    return x * K.tanh(K.softplus(x))
def get_custom_objects():
    '''
    form up a custom_objects dict so that the customized
    layer/function call could be correctly parsed when keras
    .h5 model is loading or converting
    '''
    custom_objects_dict = {
        'tf': tf,
        'mish': mish
    }

    return custom_objects_dict


In [45]:
from tensorflow.keras.models import load_model
custom_object_dict = get_custom_objects()
model_path = r'C:\Users\xia\Documents\codes\20210403_目标检测\target_detection\2_yolov4\yolov4_pycharm\weights\yolov4.h5'
model = load_model(model_path, compile=False, custom_objects=custom_object_dict)
model_format = 'H5'
K.set_learning_phase(0)


Instructions for updating:
Simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


In [46]:
prediction = model.predict([image_data])

In [48]:
prediction

AttributeError: 'list' object has no attribute 'shape'